In [5]:
!pip install -q spacy pandas unidecode
!python -m spacy download es_core_news_lg


     ---------------------------------------- 0.0/568.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/568.0 MB ? eta -:--:--
     -------------------------------------- 0.0/568.0 MB 165.2 kB/s eta 0:57:19
     -------------------------------------- 0.1/568.0 MB 327.7 kB/s eta 0:28:54
     ---------------------------------------- 0.6/568.0 MB 3.6 MB/s eta 0:02:37
     --------------------------------------- 2.3/568.0 MB 10.2 MB/s eta 0:00:56
     --------------------------------------- 4.5/568.0 MB 17.1 MB/s eta 0:00:33
     --------------------------------------- 6.3/568.0 MB 20.1 MB/s eta 0:00:29
      -------------------------------------- 7.9/568.0 MB 22.0 MB/s eta 0:00:26
      -------------------------------------- 9.2/568.0 MB 22.6 MB/s eta 0:00:25
      ------------------------------------- 10.5/568.0 MB 38.6 MB/s eta 0:00:15
      ------------------------------------- 12.4/568.0 MB 38.6 MB/s eta 0:00:15
      ------------------------------------- 14.4/568.0

In [6]:
import pandas as pd
import numpy as np
import re

from unidecode import unidecode
import spacy

# Load Spanish large model
nlp = spacy.load("es_core_news_lg")


In [7]:
train = pd.read_csv("C:/Users/Rodri/OneDrive/Rodri estudioso/GitHub/taller_3/stores/train.csv")
test  = pd.read_csv("C:/Users/Rodri/OneDrive/Rodri estudioso/GitHub/taller_3/stores/test.csv")

train.head()


,property_id,city,price,month,year,surface_total,surface_covered,rooms,bedrooms,bathrooms,property_type,operation_type,lat,lon,title,description
0,4fa04a187235d7509a43a8cf,Bogotá D.C,580000000.0,10,2020,NaN,NaN,3.0,3,NaN,Casa,Venta,4.707211,-74.099645,de casa en los cerezos,cod 2901fr461 de casa multifamiliar 4 pisos ga...
1,f9cfbb306af543a759c1f3f2,Bogotá D.C,600000000.0,10,2020,NaN,NaN,9.0,9,NaN,Casa,Venta,4.612200,-74.131642,casa en floralia bogota,casa en o en el barrio floralia en la ciudad d...
2,957a84e8e93ae2b33bc6f61b,Bogotá D.C,400000000.0,10,2020,NaN,NaN,8.0,8,NaN,Casa,Venta,4.586620,-74.138664,de casa en el barrio fatima,de casa de 7 mt2 de frente por 20 mt2 de fondo...
3,eb32d75b32bc5e8677207fc7,Bogotá D.C,800000000.0,10,2020,NaN,NaN,6.0,6,NaN,Casa,Venta,4.678147,-74.142154,casa fontibon a media cuadra de avda ferrocarril,oportunidad salio a la una amplia casa rentabl...
4,880a6e7bd74dc2568940d275,Bogotá D.C,480000000.0,10,2020,NaN,NaN,NaN,3,3.0,Casa,Venta,4.756503,-74.051553,casa en bogota villa del prado,casa villa del prado amplia estilo asturiana m...


In [8]:
## Limpiamos el texto para el patrón regex

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = unidecode(text)  # remove accents
    # replace non-alphanumeric by space
    text = re.sub(r"[^0-9a-zA-Z]+", " ", text)
    # collapse whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

for df in (train, test):
    df["description_raw"] = df["description"].fillna("")
    df["description_clean"] = df["description_raw"].apply(clean_text)


In [9]:
## Corremos spacy en el raw

def spacy_tokenize(df, text_col="description_raw", id_col="property_id"):
    """
    Run spaCy on a dataframe column and return a tidy token table.
    """
    rows = []
    texts = df[text_col].fillna("").tolist()
    ids   = df[id_col].tolist()

    for prop_id, doc in zip(ids, nlp.pipe(texts, batch_size=64, n_process=1)):
        sent_id = 0
        for sent in doc.sents:
            for token in sent:
                rows.append({
                    "property_id": prop_id,
                    "sent_id": sent_id,
                    "token": token.text,
                    "lemma": token.lemma_,
                    "pos": token.pos_,
                    "ent_type": token.ent_type_
                })
            sent_id += 1

    return pd.DataFrame(rows)

train_tokens = spacy_tokenize(train, text_col="description_raw", id_col="property_id")
train_tokens.head()

test_tokens = spacy_tokenize(test, text_col="description_raw", id_col="property_id")


In [10]:
train_tokens["lemma"].value_counts().head(50)



lemma
de               149892
y                 99015
con               98852
el                90817
en                73818
bano              39954
uno               38583
zona              38543
apartamento       34172
a                 33762
piso              31463
sala              29738
comedor           26892
cocina            26160
dos               24233
para              23700
2                 22064
amplio            19824
br                19802
contar            19236
parqueadero       19196
habitación        19190
social            17888
excelente         17771
3                 17263
alcoba            16909
cerca             14892
estudio           14298
principal         14028
independiente     13929
mucho             13711
deposito          13576
tres              13482
bao               13296
ubicado           12760
garaje            12737
casa              12632
él                12309
acceso            11844
integral          11809
terraza           11615
edificio  

In [11]:
mask_noun = train_tokens["pos"] == "NOUN"
train_tokens.loc[mask_noun, "lemma"].value_counts().head(50)


lemma
zona           36298
apartamento    33698
piso           27433
sala           24796
bano           20529
cocina         19776
habitación     19190
parqueadero    18401
alcoba         15692
acceso         11796
garaje         11479
casa           10970
terraza        10758
edificio       10707
parque         10298
deposito        9811
nivel           9023
estudio         8638
centro          8604
chimenea        8528
espacio         7395
via             6966
calle           6906
ropa            6805
sector          6686
vista           6650
conjunto        6632
bbq             6514
gas             6433
madera          6250
ascensor        5798
vigilancia      5597
visitante       5489
servicio        5483
area            5058
comedor         4903
gimnasio        4541
cuarto          4462
transporte      4258
lavanderia      4230
patio           4188
barrio          3994
carrera         3903
hora            3766
cancha          3619
bao             3617
seguridad       3557
acabado

In [12]:
train_tokens.loc[train_tokens["pos"] == "ADJ", "lemma"].value_counts().head(50)


lemma
amplio           19806
excelente        17758
comedor          15873
principal        14025
independiente    13717
ubicado          12752
integral         11617
comercial        11165
social           10231
privado          10187
exterior          7826
abierto           7675
cubierto          7671
buen              7308
comunal           6511
cerrado           6494
natural           6419
iluminado         5902
hermoso           5847
infantil          5534
segundo           5308
cuarto            5275
remodelado        4808
primero           4804
laminado          4709
residencial       4658
verde             4341
cocín             4050
construido        3950
lindo             3776
moderno           3680
espectacular      3408
gimnasio          3158
americano         3053
gran              2797
interior          2777
publico           2657
cercano           2654
exclusivo         2587
tercer            2514
mayor             2466
tranquilo         2459
apto              2379
auxil

In [13]:
mask_adj = train_tokens["pos"] == "ADJ"
train_tokens.loc[mask_adj, "lemma"].value_counts().head(50)


lemma
amplio           19806
excelente        17758
comedor          15873
principal        14025
independiente    13717
ubicado          12752
integral         11617
comercial        11165
social           10231
privado          10187
exterior          7826
abierto           7675
cubierto          7671
buen              7308
comunal           6511
cerrado           6494
natural           6419
iluminado         5902
hermoso           5847
infantil          5534
segundo           5308
cuarto            5275
remodelado        4808
primero           4804
laminado          4709
residencial       4658
verde             4341
cocín             4050
construido        3950
lindo             3776
moderno           3680
espectacular      3408
gimnasio          3158
americano         3053
gran              2797
interior          2777
publico           2657
cercano           2654
exclusivo         2587
tercer            2514
mayor             2466
tranquilo         2459
apto              2379
auxil

In [ ]:
import re
import numpy as np
import pandas as pd

# ==========
# DICCIONARIO DE LEMMAS
# ==========

PROPERTY_TYPE = {
    "apartment": ["apartamento", "apartaestudio", "duplex", "loft", "penthouse"],
    "house": ["casa"],
}

ROOM_WORDS      = ["habitacion", "alcoba", "cuarto", "dormitorio", "habitaciones"]
BATH_WORDS      = ["bano", "baos", "banos" ]
GARAGE_WORDS    = ["garaje", "parqueadero"]
GARAGE_ADJ      = ["cubierto", "independiente", "cerrado"]
STORAGE_WORDS   = ["deposito", "bodega"]
TERRACE_WORDS   = ["terraza"]
BALCONY_WORDS   = ["balcon"]
KITCHEN_WORDS   = ["cocina"]
KITCHEN_ADJ     = ["integral", "abierto"]
LAUNDRY_WORDS   = ["ropa", "lavandera", "lavanderia"]
SECURITY_WORDS  = ["seguridad", "vigilancia", "vigilante", "porteria", "portería", "portero"]

# Proximidad centrada en centros comerciales (no uso comercial)
CENTRO_COMERCIAL_PATTERNS = [
    r"\bcentro comercial\b",
    r"\bcentros comerciales\b",
]

# Uso comercial (del inmueble)
USO_COMERCIAL_PATTERNS = [
    r"\buso comercial\b",
    r"\blocal comercial(es)?\b",
    r"\barea comercial\b",
    r"\bzonas? comerciales?\b",
    r"\bsector comercial\b",
    r"\binmueble comercial\b",
]

regex_cc  = re.compile("|".join(CENTRO_COMERCIAL_PATTERNS))
regex_uso = re.compile("|".join(USO_COMERCIAL_PATTERNS), flags=re.IGNORECASE)

# ==========
# NÚMEROS EN ESPAÑOL BÁSICOS
# ==========

SPANISH_NUM_WORDS = {
    "uno": 1, "una": 1, "un": 1,
    "dos": 2,
    "tres": 3,
    "cuatro": 4,
    "cinco": 5,
    "seis": 6,
    "siete": 7,
    "ocho": 8,
    "nueve": 9,
    "diez": 10,
}

def token_to_int(tok: str):
    """Convierte un token como '2' o 'dos' en int. Si no puede, devuelve None."""
    if tok is None:
        return None
    tok = tok.lower()
    tok = tok.strip()
    if tok.isdigit():
        return int(tok)
    return SPANISH_NUM_WORDS.get(tok)



In [64]:
def add_token_index(tokens: pd.DataFrame) -> pd.DataFrame:
    tokens = tokens.sort_values(["property_id", "sent_id"]).copy()
    tokens["tok_id"] = tokens.groupby("property_id").cumcount()
    return tokens

train_tokens = add_token_index(train_tokens)
test_tokens  = add_token_index(test_tokens)


In [65]:
## Funciones de length
def add_length_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["n_palabras_desc"] = df["description_clean"].fillna("").str.split().str.len()
    df["n_palabras_title"] = df["title"].fillna("").str.split().str.len()
    
    # Versión sin códigos tipo "cod 1234"
    title_nocod = df["title"].fillna("").str.lower()
    title_nocod = title_nocod.str.replace(r"\bcod(igo)?\s*\d+\b", "", regex=True)
    title_nocod = title_nocod.str.replace(r"\s+", " ", regex=True).str.strip()
    df["n_palabras_title_nocod"] = title_nocod.str.split().str.len()
    return df


In [66]:
def add_property_type_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(list)
    
    def classify_type(lemmas):
        lemmas = set(lemmas)
        is_ap = any(l in lemmas for l in PROPERTY_TYPE["apartment"])
        is_house = any(l in lemmas for l in PROPERTY_TYPE["house"])
        if is_ap and not is_house:
            return "apartment"
        elif is_house and not is_ap:
            return "house"
        elif is_ap and is_house:
            return "ambiguous"
        else:
            return "unknown"
    
    tipo = lemmas_by_prop.apply(classify_type)
    df = df.merge(tipo.rename("tipo_inmueble_text"), on="property_id", how="left")
    return df


In [67]:
# Sala comedor habitaciones baños

def add_room_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # Flags sala / comedor
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(list)
    
    df["tiene_sala"] = df["property_id"].map(
        lambda pid: int("sala" in set(lemmas_by_prop.get(pid, [])))
    )
    df["tiene_comedor"] = df["property_id"].map(
        lambda pid: int("comedor" in set(lemmas_by_prop.get(pid, [])))
    )
    
    # sala comedor conj.
    def has_sala_comedor(text):
        if pd.isna(text):
            return 0
        t = text.lower()
        return int("sala comedor" in t or "salacomedor" in t or "sala-comedor" in t)
    
    df["sala_comedor_conjunto"] = df["description_raw"].apply(has_sala_comedor)
    
    # Conteo aproximado de habitaciones/baños usando NUM + lemma
    tokens_rooms = tokens[tokens["lemma"].isin(ROOM_WORDS + BATH_WORDS)].copy()
    
    # Juntamos el token previo como posible número
    num_candidates = tokens[["property_id", "tok_id", "token"]].copy()
    num_candidates["num_value"] = num_candidates["token"].map(token_to_int)
    
    merged = tokens_rooms.merge(
        num_candidates,
        left_on=["property_id", "tok_id"],
        right_on=["property_id", "tok_id"],
        suffixes=("", "_self"),
        how="left",
    )
    # número inmediatamente anterior
    prev = num_candidates.copy()
    prev["tok_id"] = prev["tok_id"] + 1  # para hacer join con el anterior
    prev = prev.rename(columns={"num_value": "num_prev"})
    
    merged = merged.merge(
        prev[["property_id", "tok_id", "num_prev"]],
        on=["property_id", "tok_id"],
        how="left",
    )
    
    def pick_num(row):
        if not np.isnan(row.get("num_prev", np.nan)):
            return row["num_prev"]
        return row.get("num_value", np.nan)
    
    merged["num_detected"] = merged.apply(pick_num, axis=1)
    
    # habitaciones: ROOM_WORDS
    mask_room = merged["lemma"].isin(ROOM_WORDS)
    room_counts = merged[mask_room].groupby("property_id")["num_detected"].sum(min_count=1)
    
    # si no hay número detectado, al menos poner 1 si se mencionan habitaciones
    freq_room = tokens[tokens["lemma"].isin(ROOM_WORDS)].groupby("property_id").size()
    
    n_rooms = room_counts.combine(freq_room, func=lambda n, f: n if not np.isnan(n) else float(f) if not pd.isna(f) else np.nan)
    
    # baños
    mask_bath = merged["lemma"].isin(BATH_WORDS)
    bath_counts = merged[mask_bath].groupby("property_id")["num_detected"].sum(min_count=1)
    freq_bath = tokens[tokens["lemma"].isin(BATH_WORDS)].groupby("property_id").size()
    n_baths = bath_counts.combine(freq_bath, func=lambda n, f: n if not np.isnan(n) else float(f) if not pd.isna(f) else np.nan)
    
    df = df.merge(n_rooms.rename("n_habitaciones_text"), on="property_id", how="left")
    df = df.merge(n_baths.rename("n_banos_text"), on="property_id", how="left")
    
    return df


In [68]:
# Garaje parqueo
def add_garage_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # ¿aparece alguna palabra de garaje?
    has_garage = (
        tokens[tokens["lemma"].isin(GARAGE_WORDS)]
        .groupby("property_id")
        .size()
        .gt(0)
        .astype(int)
    )
    df = df.merge(has_garage.rename("tiene_garaje_text"), on="property_id", how="left")
    df["tiene_garaje_text"] = df["tiene_garaje_text"].fillna(0).astype(int)
    
    # Conteo aproximado de # garajes usando números
    tokens_garage = tokens[tokens["lemma"].isin(GARAGE_WORDS)].copy()
    num_candidates = tokens[["property_id", "tok_id", "token"]].copy()
    num_candidates["num_value"] = num_candidates["token"].map(token_to_int)
    
    merged = tokens_garage.merge(
        num_candidates,
        on=["property_id", "tok_id"],
        how="left",
        suffixes=("", "_self"),
    )
    
    prev = num_candidates.copy()
    prev["tok_id"] = prev["tok_id"] + 1
    prev = prev.rename(columns={"num_value": "num_prev"})
    
    merged = merged.merge(
        prev[["property_id", "tok_id", "num_prev"]],
        on=["property_id", "tok_id"],
        how="left",
    )
    
    def pick_num(row):
        if not np.isnan(row.get("num_prev", np.nan)):
            return row["num_prev"]
        return row.get("num_value", np.nan)
    
    merged["num_detected"] = merged.apply(pick_num, axis=1)
    
    n_garages = merged.groupby("property_id")["num_detected"].sum(min_count=1)
    df = df.merge(n_garages.rename("n_garajes_text"), on="property_id", how="left")
    
    # Flags de cubierto / independiente
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(set)
    
    df["garaje_cubierto_text"] = df["property_id"].map(
        lambda pid: int(
            ("cubierto" in lemmas_by_prop.get(pid, set()))
            and df.loc[df["property_id"] == pid, "tiene_garaje_text"].iloc[0] == 1
        )
    )
    df["garaje_indep_text"] = df["property_id"].map(
        lambda pid: int("independiente" in lemmas_by_prop.get(pid, set()))
    )
    
    return df


In [69]:
def add_storage_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    has_storage = (
        tokens[tokens["lemma"].isin(STORAGE_WORDS)]
        .groupby("property_id")
        .size()
        .gt(0)
        .astype(int)
    )
    df = df.merge(has_storage.rename("tiene_deposito_bodega"), on="property_id", how="left")
    df["tiene_deposito_bodega"] = df["tiene_deposito_bodega"].fillna(0).astype(int)
    return df


In [70]:
def add_terrace_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(set)
    
    df["tiene_terraza"] = df["property_id"].map(
        lambda pid: int(any(l in lemmas_by_prop.get(pid, set()) for l in TERRACE_WORDS))
    )
    df["tiene_balcon"] = df["property_id"].map(
        lambda pid: int(any(l in lemmas_by_prop.get(pid, set()) for l in BALCONY_WORDS))
    )
    
    # terraza propia / privada (muy básico)
    def terraza_propia(pid):
        lem = lemmas_by_prop.get(pid, set())
        return int("terraza" in lem and ("privado" in lem or "propio" in lem))
    
    df["terraza_propia"] = df["property_id"].map(terraza_propia)
    return df


In [71]:
def add_kitchen_laundry_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(set)
    
    df["tiene_cocina"] = df["property_id"].map(
        lambda pid: int("cocina" in lemmas_by_prop.get(pid, set()))
    )
    
    def cocina_integral(pid):
        lem = lemmas_by_prop.get(pid, set())
        return int("cocina" in lem and "integral" in lem)
    
    def cocina_abierta(pid):
        lem = lemmas_by_prop.get(pid, set())
        return int("cocina" in lem and "abierto" in lem)
    
    df["cocina_integral"] = df["property_id"].map(cocina_integral)
    df["cocina_abierta"]  = df["property_id"].map(cocina_abierta)
    
    # Patio / zona de ropas: heurística simple
    def tiene_laundry(text):
        if pd.isna(text):
            return 0
        t = text.lower()
        if any(p in t for p in ["zona de ropas", "patio de ropas", "lavanderia", "lavandera"]):
            return 1
        return 0
    
    df["tiene_patio_ropas"] = df["description_raw"].apply(tiene_laundry)
    return df


In [72]:
def add_remodel_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    def flag_remodel(text):
        if pd.isna(text):
            return 0
        t = text.lower()
        # positivo
        if any(p in t for p in ["recien remodelado", "recien remodelada", "remodelado", "remodelada", "actualizado"]):
            # pero no marcar si solo dice "para remodelar"
            if "para remodelar" in t:
                return 0
            return 1
        # explícitamente "para remodelar" = 0 pero podrías guardar otra flag
        return 0
    
    df["remodelada_text"] = df["description_raw"].apply(flag_remodel)
    return df


In [73]:
def add_security_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(set)
    
    def has_security(pid):
        lem = lemmas_by_prop.get(pid, set())
        if any(w in lem for w in SECURITY_WORDS):
            return 1
        return 0
    
    df["tiene_vigilancia_text"] = df["property_id"].map(has_security)
    return df


In [74]:
def add_commercial_features(df: pd.DataFrame, text_col: str = "description_clean") -> pd.DataFrame:
    df = df.copy()
    text = df[text_col].fillna("")
    
    df["menciona_centro_comercial"] = text.apply(
        lambda t: int(bool(regex_cc.search(t)))
    )
    df["uso_comercial_text"] = text.apply(
        lambda t: int(bool(regex_uso.search(t)))
    )
    return df


In [75]:
def add_proximity_features(df: pd.DataFrame, text_col: str = "description_clean") -> pd.DataFrame:
    df = df.copy()
    txt = df[text_col].fillna("").str.lower()
    
    df["menciona_cercania"] = txt.apply(
        lambda t: int(any(p in t for p in ["cerca a", "cerca de", "frente a", "a pocos minutos"]))
    )
    
    df["cercania_bus"] = txt.apply(
        lambda t: int(any(p in t for p in ["transmilenio", "estacion", "estación", "paradero", "autopista norte", "transporte publico"]))
    )
    
    df["cercania_cc"] = df["menciona_centro_comercial"]  # de la función anterior
    
    df["cercania_cafe"] = txt.apply(
        lambda t: int(any(p in t for p in ["cafe", "café", "restaurante", "bares"]))
    )
    
    # 'a X cuadras'
    df["menciona_cuadras"] = txt.apply(
        lambda t: int(bool(re.search(r"a\s+\d+\s+cuadras", t)))
    )
    
    return df


In [76]:
def add_text_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    """
    df: DataFrame con columnas:
        - property_id
        - title
        - description_raw
        - description_clean
    tokens: DataFrame con columnas:
        - property_id
        - sent_id
        - tok_id
        - token
        - lemma
        - pos
    """
    out = df.copy()
    out = add_length_features(out)
    out = add_property_type_features(out, tokens)
    out = add_room_features(out, tokens)
    out = add_garage_features(out, tokens)
    out = add_storage_features(out, tokens)
    out = add_terrace_features(out, tokens)
    out = add_kitchen_laundry_features(out, tokens)
    out = add_remodel_features(out)
    out = add_security_features(out, tokens)
    out = add_commercial_features(out, text_col="description_clean")
    out = add_proximity_features(out, text_col="description_clean")
    return out


In [77]:
train_enriched = add_text_features(train, train_tokens)
test_enriched  = add_text_features(test,  test_tokens)



In [29]:
train_enriched.columns


Index(['property_id', 'city', 'price', 'month', 'year', 'surface_total',
       'surface_covered', 'rooms', 'bedrooms', 'bathrooms', 'property_type',
       'operation_type', 'lat', 'lon', 'title', 'description',
       'description_raw', 'description_clean', 'n_palabras_desc',
       'n_palabras_title', 'n_palabras_title_nocod', 'tipo_inmueble_text',
       'tiene_sala', 'tiene_comedor', 'sala_comedor_conjunto',
       'n_habitaciones_text', 'n_banos_text', 'tiene_garaje_text',
       'n_garajes_text', 'garaje_cubierto_text', 'garaje_indep_text',
       'tiene_deposito_bodega', 'tiene_terraza', 'tiene_balcon',
       'terraza_propia', 'tiene_cocina', 'cocina_integral', 'cocina_abierta',
       'tiene_patio_ropas', 'remodelada_text', 'tiene_vigilancia_text',
       'menciona_centro_comercial', 'uso_comercial_text', 'menciona_cercania',
       'cercania_bus', 'cercania_cc', 'cercania_cafe', 'menciona_cuadras'],
      dtype='object')

In [78]:
train_enriched["tiene_balcon"].value_counts(dropna=False)


tiene_balcon
0    31784
1     6860
Name: count, dtype: int64

In [30]:
cols_new = [
    c for c in train_enriched.columns 
    if c.startswith("tiene_") 
    or c.startswith("n_") 
    or c.endswith("_text") 
    or c.endswith("_propia")
]

for col in cols_new:
    print(f"\n📌 {col}")
    print(train_enriched[col].value_counts(dropna=False))



📌 n_palabras_desc
n_palabras_desc
28     1114
54      696
62      652
38      646
56      615
       ... 
273       1
308       1
231       1
393       1
334       1
Name: count, Length: 304, dtype: int64

📌 n_palabras_title
n_palabras_title
4     9173
5     5648
6     5248
7     5086
3     5084
8     4192
9     2070
10     936
2      603
11     275
12      97
1       74
13      39
0       25
15      23
14      20
16      16
17      12
18      11
19       7
20       2
21       2
22       1
Name: count, dtype: int64

📌 n_palabras_title_nocod
n_palabras_title_nocod
4     9175
5     5650
6     5248
7     5086
3     5084
8     4190
9     2069
10     935
2      603
11     275
12      97
1       74
13      39
0       25
15      23
14      20
16      16
17      12
18      11
19       7
20       2
21       2
22       1
Name: count, dtype: int64

📌 tipo_inmueble_text
tipo_inmueble_text
apartment    25667
house         6988
unknown       3906
ambiguous     2071
NaN             12
Name: count, d

In [79]:
import os

train_enriched.to_csv("train_enriched2.csv", index=False, encoding="utf-8")
test_enriched.to_csv("test_enriched2.csv", index=False, encoding="utf-8")

